In [1]:
from coffea.nanoevents import NanoEventsFactory, PFNanoAODSchema
import awkward as ak
import warnings
import fastjet
from variable_functions import *
import scipy
import json

/afs/crc.nd.edu/user/c/cmoore24/Public/hgg/standalone_notebooks/skims/singlemuon/variable_functions.py:3: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  import coffea.nanoevents.methods.vector as vector


In [2]:
warnings.filterwarnings('ignore', 'Found duplicate')
warnings.filterwarnings('ignore', 'Missing cross')

In [3]:
path = '/project01/ndcms/cmoore24/samples/diboson/zz/nano_mc2017_6.root'

In [4]:
events = NanoEventsFactory.from_root(
    {f'{path}':'/Events'},
    delayed=True,
    schemaclass=PFNanoAODSchema,
    metadata={'dataset':'test'},
).events()

In [5]:
goodmuon = (
            (events.Muon.pt > 10)
            & (abs(events.Muon.eta) < 2.4)
            & (events.Muon.pfRelIso04_all < 0.25) # invert the isolation cut; > 0.25, check for QCD (maybe try > 1.0)
            & events.Muon.looseId
        )

nmuons = ak.sum(goodmuon, axis=1)
leadingmuon = ak.firsts(events.Muon[goodmuon])

goodelectron = (
    (events.Electron.pt > 10)
    & (abs(events.Electron.eta) < 2.5)
    & (events.Electron.cutBased >= 2) #events.Electron.LOOSE
)
nelectrons = ak.sum(goodelectron, axis=1)

ntaus = ak.sum(
    (
        (events.Tau.pt > 20)
        & (abs(events.Tau.eta) < 2.3)
        & (events.Tau.rawIso < 5)
        & (events.Tau.idDeepTau2017v2p1VSjet)
        & ak.all(events.Tau.metric_table(events.Muon[goodmuon]) > 0.4, axis=2)
        & ak.all(events.Tau.metric_table(events.Electron[goodelectron]) > 0.4, axis=2)
    ),
    axis=1,
)

nolepton = ((nmuons == 0) & (nelectrons == 0) & (ntaus == 0))

onemuon = ((nmuons == 1) & (nelectrons == 0) & (ntaus == 0))

In [6]:
ak.fill_none(events.FatJet.delta_r(events.FatJet.nearest(events.Muon[goodmuon], axis=1)) > 0.8, True).compute()

<Array [[], [], [], [], [], ..., [], [], [], [], []] type='335000 * var * bool'>

In [7]:
fatjetSelect = (
            (events.FatJet.pt >= 450)
            & (events.FatJet.pt <= 1200)
            & (abs(events.FatJet.eta) <= 2.4)
            & (events.FatJet.msoftdrop >= 40)
            & (events.FatJet.msoftdrop <= 200)
            & (onemuon)
            & (ak.fill_none(events.FatJet.delta_r(events.FatJet.nearest(events.Muon[goodmuon], axis=1)) > 0.8, True))
            # & (trigger)
            # & (events.FatJet.btag_count == 0)
        )

In [8]:
events["goodjets"] = events.FatJet[fatjetSelect]
mask = ~ak.is_none(ak.firsts(events.goodjets))
events = events[mask]

In [9]:
events.goodjets.compute()

<FatJetArray [[{area: 2.06, ...}], ..., [{...}]] type='10 * var * FatJet[ar...'>

In [39]:
pf = ak.flatten(events.goodjets.constituents.pf, axis=1)
pf['px'] = pf.px
pf['py'] = pf.py
pf['pz'] = pf.pz
pf['E'] = pf.E

In [40]:
jetdef = fastjet.JetDefinition(
    fastjet.cambridge_algorithm, 1.0
)

In [41]:
cluster = fastjet.ClusterSequence(pf, jetdef)

In [34]:
pf.px.compute()

<Array [[-3.73, -18.4, ..., -1.14, -1.13], ...] type='11 * option[var * ?fl...'>